## DGA 데이터 전처리 : Part 3 날짜 정보

<div style="text-align: right"> 작성자 : 유광명</div>  

<div style="text-align: right"> 1차 수정일: 2020.05.12 </div> 
<div style="text-align: right"> 최종 수정일: 2023.02.01 </div> 

### 0. 환경설정 및 데이터 로딩

In [1]:
from tqdm.notebook import tqdm
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import seaborn as sns
import scipy as sp
import datetime
import missingno as msno
import warnings; warnings.filterwarnings('ignore')
plt.style.use('ggplot')
%matplotlib inline

# Options for pandas
pd.options.display.max_columns = 60

mpl.rcParams['axes.unicode_minus'] = False
_path = "c:\Windows\Fonts\KoPubWorld Dotum Bold.ttf"
_font_name = font_manager.FontProperties(fname=_path).get_name()
rc('font', family=_font_name)
mpl.rcParams['axes.unicode_minus']=False # 마이너스 폰트 깨짐방지

# 그래프 크기 설정
font_n = 15
font_d = 12
figure_hrz = 15

In [2]:
inputPath = 'result2.pkl'

In [3]:
df = pd.read_pickle(inputPath)

In [4]:
df.head(2)

,office1,office2,name,S/S,eq_type,eq_num,eq_phase,maker,volt,volt_by_name,mfg_year,sample_date,operation,anal_type,H2,C2H2,C2H4,CH4,C2H6,C3H8,TCG,r1(KEPCO),CO,CO2,r2(KEPCO),etc
0,제주본부,제주전력지사,신서귀S/S 154kV #1 M.Tr A,신서귀S/S,M.Tr,#1,A,효성,154,154,1998.0,20170111,운전중,임시,0.0,0.0,3.0,0.0,1.0,1.0,5.0,정상,39.0,543.0,정상,정밀점검 1개월후
1,제주본부,제주전력지사,신서귀S/S 154kV #1 M.Tr A,신서귀S/S,M.Tr,#1,A,효성,154,154,1998.0,20171103,운전중,임시,13.0,0.0,2.0,1.0,1.0,2.0,19.0,정상,117.0,1600.0,정상,중부하 운전 변압기


### 3. 날짜 정보 컬럼 전처리  
- 날짜 정보 컬럼 : mfg_year, sample_date

In [5]:
df.columns

Index(['office1', 'office2', 'name', 'S/S', 'eq_type', 'eq_num', 'eq_phase',
       'maker', 'volt', 'volt_by_name', 'mfg_year', 'sample_date', 'operation',
       'anal_type', 'H2', 'C2H2', 'C2H4', 'CH4', 'C2H6', 'C3H8', 'TCG',
       'r1(KEPCO)', 'CO', 'CO2', 'r2(KEPCO)', 'etc'],
      dtype='object')

#### 3.1 mfg_year(제작년)  
- 자료형(float64)을 int로 수정
- 결측치 처리 : 일단 0으로 대체
- **향 후:** 결측치 처리방안 확정

In [6]:
df[['mfg_year']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135942 entries, 0 to 135941
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   mfg_year  135801 non-null  float64
dtypes: float64(1)
memory usage: 1.0 MB


In [7]:
df['mfg_year'].isnull().sum()
#Null로 두면 자료형 변환이 안됨. 추후 결측치 처리 필요

141

In [8]:
df.loc[df['mfg_year'].isnull(), 'mfg_year'] = 0 
# 일단 0으로 대체
df['mfg_year'] = df['mfg_year'].astype('uint32')

In [9]:
df['mfg_year'].describe()

count    135942.000000
mean       1997.236307
std          64.837550
min           0.000000
25%        1995.000000
50%        1999.000000
75%        2004.000000
max        2020.000000
Name: mfg_year, dtype: float64

In [10]:
df['mfg_year'].dtype

dtype('uint32')

#### 3.2 sample-date(분석일자)  
- 이상치 제거완료

1) 이상치 제거

In [11]:
# 오늘 날짜보다 하루 뒤에 측정된 것으로 확인되는 것을 결측치로 보고 검출
th = int(datetime.datetime.today().strftime('%Y%m%d'))+1
df = df.drop(index = df[df['sample_date']>th].index.tolist()[0]) 
# 해당 컬럼 제거

In [12]:
df[df['sample_date']> th]

,office1,office2,name,S/S,eq_type,eq_num,eq_phase,maker,volt,volt_by_name,mfg_year,sample_date,operation,anal_type,H2,C2H2,C2H4,CH4,C2H6,C3H8,TCG,r1(KEPCO),CO,CO2,r2(KEPCO),etc


2) sample_year, sample_month, sample_season 컬럼 생성  

In [13]:
df['sample_date']= pd.to_datetime(df['sample_date'], format = '%Y%m%d')
df['sample_year'] = df['sample_date'].dt.year
df['sample_month'] = df['sample_date'].dt.month

In [14]:
def to_season(month):
    if month in [3, 4, 5]: 
        return 0
    if month in [6, 7, 8]:
        return 1
    if month in [9, 10, 11]:
        return 2
    if month in [12, 1, 2]:
        return 3

In [15]:
df['sample_season'] = df['sample_month'].apply(to_season)

3) age(year), age(month) 컬럼 생성  
- 생성 후 age 값이 음수로 계산되는 부분 삭제(1275건)
- **향 후:** mfg 컬럼 신뢰성 검토, 활용부서에 변압기 정보 데이터 요청

In [16]:
df['age(year)'] = (df['sample_year']-df['mfg_year'])
df['age(month)'] = (df['sample_year']-df['mfg_year'])*12 + df['sample_month']-1

In [17]:
df[(df['age(year)'] < 0)&(df['age(month)'] < 0)].shape

(1275, 31)

In [18]:
before_size = df.shape[0]
print("- before size: {}".format(before_size))

df = df[(df['age(year)'] >= 0) & (df['age(month)'] >= 0)]
df = df.reset_index(drop = True, inplace=False)

after_size = df.shape[0]
print("- after size: {}".format(after_size))
print('제거한 샘플 수: {}'.format(before_size-after_size))

- before size: 135941
- after size: 134666
제거한 샘플 수: 1275


In [19]:
df.shape

(134666, 31)

4) long term 컬럼 생성
- 3년 이상 운전 1, 3년 이내 운전 0  
- 함수수정(5.26) : age(year) 대신 ageMonth를 인자로 사용

In [20]:
def check_age(ageMonth):
    if ageMonth < 36:
        return 0
    else:
        return 1

In [21]:
df['long_term'] = df['age(month)'].apply(check_age)

#### 완료 및 중간정리

In [22]:
df.columns

Index(['office1', 'office2', 'name', 'S/S', 'eq_type', 'eq_num', 'eq_phase',
       'maker', 'volt', 'volt_by_name', 'mfg_year', 'sample_date', 'operation',
       'anal_type', 'H2', 'C2H2', 'C2H4', 'CH4', 'C2H6', 'C3H8', 'TCG',
       'r1(KEPCO)', 'CO', 'CO2', 'r2(KEPCO)', 'etc', 'sample_year',
       'sample_month', 'sample_season', 'age(year)', 'age(month)',
       'long_term'],
      dtype='object')

In [23]:
tr_cols = ['office1', 'office2', 'name', 'S/S', 'eq_type', 'eq_num', 'eq_phase',
       'maker', 'volt', 'volt_by_name','operation', 'anal_type'] #11
date_cols = ['mfg_year', 'sample_date',  'sample_year','sample_month', 
             'sample_season', 'age(year)', 'age(month)','long_term'] #
gas_cols = [ 'H2', 'C2H2', 'C2H4', 'CH4', 'C2H6', 'C3H8', 'TCG','CO', 'CO2']
result_cols = ['r1(KEPCO)', 'r2(KEPCO)']
etc = ['etc']

In [24]:
set(df.columns) == set(tr_cols + date_cols + gas_cols + result_cols + etc)

True

In [25]:
df = df[tr_cols + date_cols + gas_cols + result_cols + etc]

In [29]:
df[100: 150]

,office1,office2,name,S/S,eq_type,eq_num,eq_phase,maker,volt,volt_by_name,operation,anal_type,mfg_year,sample_date,sample_year,sample_month,sample_season,age(year),age(month),long_term,H2,C2H2,C2H4,CH4,C2H6,C3H8,TCG,CO,CO2,r1(KEPCO),r2(KEPCO),etc
100,제주본부,제주전력지사,신서귀S/S 154kV #2 M.Tr B,신서귀S/S,M.Tr,#2,B,효성,154,154,운전중,NaN,1998,2006-01-02,2006,1,3,8,96,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,0.0,정상,정상,NaN
101,제주본부,제주전력지사,신서귀S/S 154kV #2 M.Tr B,신서귀S/S,M.Tr,#2,B,효성,154,154,운전중,정기,1998,2018-01-08,2018,1,3,20,240,1,67.0,0.0,9.0,7.0,38.0,30.0,151.0,76.0,4385.0,정상,정상,NaN
102,제주본부,제주전력지사,신서귀S/S 154kV #2 M.Tr B,신서귀S/S,M.Tr,#2,B,효성,154,154,운전중,정기,1998,2014-11-06,2014,11,2,16,202,1,29.0,0.0,17.0,24.0,65.0,34.0,169.0,657.0,1403.0,정상,정상,가스분석
103,제주본부,제주전력지사,신서귀S/S 154kV #2 M.Tr B,신서귀S/S,M.Tr,#2,B,효성,154,154,운전중,정기,1998,2013-12-30,2013,12,3,15,191,1,113.0,0.0,13.0,18.0,47.0,26.0,217.0,873.0,1958.0,정상,주의I,정기분석
104,제주본부,제주전력지사,신서귀S/S 154kV #2 M.Tr B,신서귀S/S,M.Tr,#2,B,효성,154,154,운전중,NaN,1998,2007-01-10,2007,1,3,9,108,1,14.0,0.0,0.0,3.0,0.0,0.0,17.0,161.0,1276.0,정상,정상,NaN
105,제주본부,제주전력지사,신서귀S/S 154kV #2 M.Tr B,신서귀S/S,M.Tr,#2,B,효성,154,154,운전중,정기,1998,2018-12-28,2018,12,3,20,251,1,13.0,0.0,11.0,15.0,59.0,37.0,135.0,150.0,628.0,정상,정상,NaN
106,제주본부,제주전력지사,신서귀S/S 154kV #2 M.Tr B,신서귀S/S,M.Tr,#2,B,효성,154,154,운전중,NaN,1998,2004-12-20,2004,12,3,6,83,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,257.0,546.0,정상,정상,NaN
107,제주본부,제주전력지사,신서귀S/S 154kV #2 M.Tr B,신서귀S/S,M.Tr,#2,B,효성,154,154,운전중,정기,1998,2009-12-28,2009,12,3,11,143,1,20.5,0.0,0.9,3.2,3.8,1.7,30.1,546.3,1388.6,정상,정상,NaN
108,제주본부,제주전력지사,신서귀S/S 154kV #2 M.Tr B,신서귀S/S,M.Tr,#2,B,효성,154,154,운전중,임시,1998,2016-11-07,2016,11,2,18,226,1,234.0,0.0,25.0,33.0,113.0,50.0,455.0,762.0,2468.0,요주의I,정상,제주전(변전)-557('16. 9.08) : 중부하운전 주변압기 절연유 가스분석 요청
109,제주본부,제주전력지사,신서귀S/S 154kV #2 M.Tr C,신서귀S/S,M.Tr,#2,C,효성,154,154,운전중,추적,1998,2013-08-01,2013,8,1,15,187,1,53.0,0.0,57.0,60.0,241.0,147.0,558.0,459.0,1652.0,요주의I,정상,요주의Ⅰ


In [27]:
df.shape

(134666, 32)

In [28]:
df.to_pickle('result3.pkl')